In [ ]:
import pyarrow.csv as pv
import pyarrow.parquet as pq
import pandas as pd
import os

In [ ]:

import requests
from bs4 import BeautifulSoup


In [ ]:
file_list = []
url_base = 'https://www.irs.gov/downloads/irs-soi?page='
for i in range(460):
    page = requests.get(f'{url_base}{i}')
    soup = BeautifulSoup(page.content, 'html.parser')
    rs = soup.find_all(headers='view-uri-table-column')
    file_list += [r.text for r in rs]

In [ ]:
https://www.irs.gov/pub/irs-soi/21eoextract990pf.
https://www.irs.gov/pub/irs-soi/16eofinextract990pf.dat

In [ ]:
for f in file_list:
    if 'extract' in f:
        print(f)

In [ ]:
def ismixed(a):
    try:
        max(a)
        return False
    except TypeError as e: # we take this to imply mixed type
        msg, fst, and_, snd = str(e).rsplit(' ', 3)
        assert msg=="'>' not supported between instances of"
        assert and_=="and"
        assert fst!=snd
        return True
    except ValueError as e: # catch empty arrays
        assert str(e)=="max() arg is an empty sequence"
        return False

In [ ]:
dirs = [
    'IRS',
    'NCCS',
]

f_exts = [
    'csv',
    'xlsx',
    'dat',
    'txt',
    'parquet'
]

for d in dirs:
    content = os.listdir(f'./{d}')
    files = {}
    for f_ext in f_exts:
        files[f_ext] = [os.path.splitext(c)[0] for c in content if c.endswith(f'.{f_ext}')]
    for f_csv in files['csv']:
        if f_csv not in files['parquet']:
            print(f'loading {f_csv}.csv')
            table = pv.read_csv(f'./{d}/{f_csv}.csv')
            print(f'saving {f_csv}.parquet')
            pq.write_table(table, f'./{d}/{f_csv}.parquet')
    for f_xlsx in files['xlsx']:
        if f_xlsx not in files['parquet']:
            print(f'loading {f_xlsx}.xlsx')
            df = pd.read_excel(f'./{d}/{f_xlsx}.xlsx')
            for c in df.columns:
                if ismixed(df[c]):
                    print(f'{c} is mixed type.  Converting to str')
                    df[c] = df[c].astype(str)
            print(f'saving {f_xlsx}.parquet')
            try:
                df.to_parquet(f'./{d}/{f_xlsx}.parquet')
            except:
                print(f'saving {f_xlsx}.parquet failed')
    for f_dat in files['dat']:
        if f_dat not in files['parquet']:
            with open(f'./{d}/{f_dat}.dat') as f:
                first_line = '\n'
                while first_line == '\n' and first_line != '':
                    first_line = f.readline()
            if '|' in first_line:
                delim = '|'
            elif ',' in first_line:
                delim = ','
            elif ' ' in first_line:
                delim = ' '
            else:
                print(f'unable to determine delimiter from {first_line}')
            print(f'loading {f_dat}.dat')
            df = pd.read_csv(f'./{d}/{f_dat}.dat', sep=delim)
            print(f'saving {f_dat}.parquet')
            df.to_parquet(f'./{d}/{f_dat}.parquet')
    for f_txt in files['txt']:
        if f_txt not in files['parquet']:
            with open(f'./{d}/{f_txt}.txt') as f:
                first_line = '\n'
                while first_line == '\n' and first_line != '':
                    first_line = f.readline()
            if '|' in first_line:
                delim = '|'
            elif ',' in first_line:
                delim = ','
            elif ' ' in first_line:
                delim = ' '
            else:
                print(f'unable to determine delimiter from {first_line}')
            print(f'loading {f_txt}.txt')
            df = pd.read_csv(f'./{d}/{f_txt}.txt', sep=delim)
            print(f'saving {f_txt}.parquet')
            df.to_parquet(f'./{d}/{f_txt}.parquet')
        

In [ ]:
files

In [ ]:
df = pd.read_csv('./IRS\data-download-epostcard.txt',
                         sep='|',
                         # on_bad_lines='skip',
                         usecols=[0, 1],
                         names=['EIN', 'EPOSTCARD'],
                         dtype='str')

In [ ]:
df.index

In [ ]:
with open('./IRS\data-download-epostcard.txt') as f:
    first_line = '\n'
    while first_line == '\n' and first_line != '':
        first_line = f.readline()

In [ ]:
'|' in first_line

In [ ]:
path = './NCCS'
files = os.listdir(path)
prefix = ['coreco.', 'nccs.']
for f in files:
    for p in prefix:
        if f.startswith(p):
            os.rename(f'{path}/{f}', f'{path}/{f.removeprefix(p)}')

In [ ]:
file = './IRS/py14_EZ.dat'

In [ ]:
df = pd.read_csv(file, sep=' ')

In [ ]:
df